In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rasterio
# import matplotlib.pyplot as plt
# from rasterio.plot import show as rio_show
# import matplotlib as mpl
import pandas as pd
import geopandas as gpd
import numpy as np

In [3]:
df = pd.read_csv('../results_2022.csv', dtype=str)
df

,Date,Chip (px),Resolution (px),Prefilter,Subpixel,Software,Vx,Vy,OIE-x,OIE-y,OI-peak-x,OI-peak-y,slope_max-normal-E,slope_min-shear-E,E_ratio,pt0_velodiff,pt1_velodiff,pt2_velodiff
0,Sen2-20180304-20180314,48,12,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.5234375,0.546875,-0.0703125,0.1953125,0.07121240778955427,0.1476625221583858,0.4822646041029315,0.21698178568504134,-0.09244126054818036,1.0609976487758717
1,Sen2-20180304-20180314,48,12,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.421875,0.4296875,-0.0625,0.1640625,0.057825730765264476,0.10187571345771824,0.5676105599914552,0.08807284413309813,-0.07274006850532372,NaN
2,Sen2-20180304-20180314,48,12,None,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.82421875,0.92578125,-0.0416666679084301,0.1354166716337204,0.08011465048394165,0.1586928351360657,0.5048410056777303,-0.09313509723984692,-0.19221693563022335,4.459473388866401
3,Sen2-20180304-20180314,48,1,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.59375,0.578125,-0.0625,0.1875,0.06251839827098954,0.12021969630375705,0.5200345716481047,0.14568437030065817,-0.09244126054818036,1.03892143835197
4,Sen2-20180304-20180314,48,1,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.40625,0.46875,-0.0625,0.15625,0.03541088154443304,0.0732625344368767,0.48334229516647687,0.033874697708833224,-0.07274006850532372,7.3252742475067825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Sen2-20180508-20180627,64,8,Gau,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.34062498807907104,0.5375000238418579,0.05000000074505806,-0.05000000074505806,0.15760919873237936,0.3327143712349554,0.4737072166356147,-0.012847301708761805,NaN,NaN
168,Sen2-20180508-20180627,32,4,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.25,0.33125001192092896,0.03750000149011612,-0.012500000186264515,0.18708346203550627,0.3905031987969051,0.4790830462129085,-0.03118168585411152,NaN,NaN
169,Sen2-20180508-20180627,32,8,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.16249999403953552,0.24687500298023224,0.03750000149011612,-0.02500000037252903,0.6872760424471992,1.2286516484977683,0.5593742077239784,-0.04018971907390767,NaN,NaN
170,Sen2-20180508-20180627,64,4,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.17499999701976776,0.2562499940395355,0.03750000149011612,-0.012500000186264515,0.17688152982057523,0.3744389532922634,0.4723908350489183,-0.057865862759106546,NaN,NaN


In [4]:
gps_files = ['/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_2018-04-05_to_2018-03-04_GPS', 
             '/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_2018-08-18_to_2018-08-02_GPS',
             '/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_2018-03-14_to_2018-03-04_GPS',
             '/home/jovyan/Projects/PX_comparison/GPS/Kaskawulsh_v2_2018-05-08_to_2018-06-27_GPS.csv']

datestrs = ['LS8-20180304-20180405', 'LS8-20180802-20180818', 'Sen2-20180304-20180314', 'Sen2-20180508-20180627']

In [5]:
for gps_file, datestr in zip(gps_files, datestrs):
    gps = pd.read_csv(gps_file) 
    gps = gpd.GeoDataFrame(gps, geometry=gpd.points_from_xy(gps['end_easting'], gps['end_northing']), crs='EPSG:32607')
    gps_xy = list(gps[['end_easting', 'end_northing']].to_records(index=False))
    if datestr == 'Sen2-20180508-20180627':
        gps_xy[-2] = (601790.43877479, 6733753.77267354)  # it should be (nan, nan), (nan, nan) but nans does not work with rio.sample
        gps_xy = gps_xy[:-1]
    print(gps_xy)
    
    df_s = df.loc[df['Date'] == datestr]
    for idx, row in df_s.iterrows():
        vx_grid = rasterio.open(row.Vx)
        vy_grid = rasterio.open(row.Vy)
        sample_gen_vx = vx_grid.sample(gps_xy)
        vx_sampled = np.array([float(record) for record in sample_gen_vx])
        sample_gen_vy = vy_grid.sample(gps_xy)
        vy_sampled = np.array([float(record) for record in sample_gen_vy])
        vx_sampled[vx_sampled < -9998] = np.nan
        vy_sampled[vy_sampled < -9998] = np.nan
        vx_sampled -= float(df.loc[idx, 'OI-peak-x'])
        vy_sampled -= float(df.loc[idx, 'OI-peak-y'])
        v_sampled = np.sqrt(vx_sampled ** 2 + vy_sampled ** 2)
        df.loc[idx, 'pt0_velodiff'] = v_sampled[0] - np.abs(gps.loc[0, 'velocity (m/d)'])
        df.loc[idx, 'pt1_velodiff'] = v_sampled[1] - np.abs(gps.loc[1, 'velocity (m/d)'])
        df.loc[idx, 'pt2_velodiff'] = v_sampled[2] - np.abs(gps.loc[2, 'velocity (m/d)'])

[(621306.41954208, 6738829.50233354), (610435.5249175, 6737129.57698521), (601733.22946583, 6733710.66504834)]
[(621363.01607688, 6738895.12164604), (610506.52739125, 6737089.56006354), (601790.43877479, 6733753.77267354)]
[(621306.41954208, 6738829.50233354), (610435.5249175, 6737129.57698521), (601733.22946583, 6733710.66504834)]
[(621348.11544947, 6738880.22819989), (610481.28682665, 6737102.95371238), (601790.43877479, 6733753.77267354)]


In [6]:
df

,Date,Chip (px),Resolution (px),Prefilter,Subpixel,Software,Vx,Vy,OIE-x,OIE-y,OI-peak-x,OI-peak-y,slope_max-normal-E,slope_min-shear-E,E_ratio,pt0_velodiff,pt1_velodiff,pt2_velodiff
0,Sen2-20180304-20180314,48,12,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.5234375,0.546875,-0.0703125,0.1953125,0.07121240778955427,0.1476625221583858,0.4822646041029315,0.070795,-0.024049,0.933118
1,Sen2-20180304-20180314,48,12,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.421875,0.4296875,-0.0625,0.1640625,0.057825730765264476,0.10187571345771824,0.5676105599914552,-0.079942,-0.001844,NaN
2,Sen2-20180304-20180314,48,12,None,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.82421875,0.92578125,-0.0416666679084301,0.1354166716337204,0.08011465048394165,0.1586928351360657,0.5048410056777303,-0.21345,-0.181267,4.574336
3,Sen2-20180304-20180314,48,1,Gau,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.59375,0.578125,-0.0625,0.1875,0.06251839827098954,0.12021969630375705,0.5200345716481047,-0.000617,-0.033971,0.922879
4,Sen2-20180304-20180314,48,1,NAOF,16-node oversampling,CARST,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,/home/jovyan/Projects/PX_comparison/PX/CARST/2...,0.40625,0.46875,-0.0625,0.15625,0.03541088154443304,0.0732625344368767,0.48334229516647687,-0.126817,-0.004175,7.474734
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Sen2-20180508-20180627,64,8,Gau,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.34062498807907104,0.5375000238418579,0.05000000074505806,-0.05000000074505806,0.15760919873237936,0.3327143712349554,0.4737072166356147,-0.00582,NaN,NaN
168,Sen2-20180508-20180627,32,4,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.25,0.33125001192092896,0.03750000149011612,-0.012500000186264515,0.18708346203550627,0.3905031987969051,0.4790830462129085,-0.048853,NaN,NaN
169,Sen2-20180508-20180627,32,8,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.16249999403953552,0.24687500298023224,0.03750000149011612,-0.02500000037252903,0.6872760424471992,1.2286516484977683,0.5593742077239784,-0.046996,NaN,NaN
170,Sen2-20180508-20180627,64,4,NAOF,pyrUP,autoRIFT,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,/home/jovyan/Projects/PX_comparison/PX/autoRIF...,0.17499999701976776,0.2562499940395355,0.03750000149011612,-0.012500000186264515,0.17688152982057523,0.3744389532922634,0.4723908350489183,-0.074091,NaN,NaN


In [7]:
df.to_csv('../results_2022.csv', index=False)

In [8]:
df.loc[idx, 'OI-peak-x']

'0.03750000149011612'